In [2]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import tensorflow as tf
# !pip install transformers
from transformers import BertTokenizer

2023-02-14 01:33:25.243454: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
# None=0, Seek=1, Provide=2

df = pd.read_csv("CSN.csv")
df

,text,label
0,I don't know who left other than him.,0
1,I also was very depressed and cryed very very ...,1
2,I am a 6 yr survivor and have been out of trea...,0
3,I have enough energy for 16 hour periods.,0
4,I was diagnosed in October 2009 at age 53.,0
...,...,...
5002,"You are a creative person, and I can just imag...",0
5003,I thought they were supposed to eliminate the ...,0
5004,"I could use a good hug too, from someone who u...",1
5005,"You take 1 pill one hour prior to treatment, 1...",0


In [4]:
df_seek = df.copy()
print(df_seek.groupby('label').describe())
df_response = df.copy()
print(df_response.groupby('label').describe())
df_seek.loc[df_seek['label'] == 2, 'label'] = 0
print(df_seek.groupby('label').describe())
df_response.loc[df_response['label'] == 1, 'label'] = 0
print(df_response.groupby('label').describe())
df_response.loc[df_response['label'] == 2, 'label'] = 1
print(df_response.groupby('label').describe())

       text                                                               
      count unique                                                top freq
label                                                                     
0      2995   2995              I don't know who left other than him.    1
1      1046   1046  I also was very depressed and cryed very very ...    1
2       966    966  I just wanted to let you know I was here if yo...    1
       text                                                               
      count unique                                                top freq
label                                                                     
0      2995   2995              I don't know who left other than him.    1
1      1046   1046  I also was very depressed and cryed very very ...    1
2       966    966  I just wanted to let you know I was here if yo...    1
       text                                                               
      count unique       

TRAINING SEEKER SIDE ON CSN

In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

In [6]:
token = tokenizer.encode_plus(
    df_seek['text'].iloc[0], 
    max_length=256, 
    truncation=True, 
    padding='max_length', 
    add_special_tokens=True,
    return_tensors='tf'
)

Metal device set to: Apple M1 Pro


2023-02-13 23:59:06.566302: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-13 23:59:06.568384: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-02-13 23:59:06.568424: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [7]:
X_input_ids = np.zeros((len(df), 256))
X_attn_masks = np.zeros((len(df), 256))

In [8]:
def generate_training_data(df_balanced, ids, masks, tokenizer):
    for i, text in tqdm(enumerate(df_seek['text'])):
        tokenized_text = tokenizer.encode_plus(
            text,
            max_length=256, 
            truncation=True, 
            padding='max_length', 
            add_special_tokens=True,
            return_tensors='tf'
        )
        ids[i, :] = tokenized_text.input_ids
        masks[i, :] = tokenized_text.attention_mask
    return ids, masks

In [9]:
X_input_ids, X_attn_masks = generate_training_data(df, X_input_ids, X_attn_masks, tokenizer)

0it [00:00, ?it/s]

In [10]:
labels = np.zeros((len(df_seek), 2))
labels.shape

(5007, 2)

In [11]:
labels[np.arange(len(df_seek)), df_seek['label'].values] = 1 # one-hot encoded target te

In [12]:
# creating a data pipeline using tensorflow dataset utility, creates batches of data for easy loading...
dataset = tf.data.Dataset.from_tensor_slices((X_input_ids, X_attn_masks, labels))
dataset.take(1) # one sample data

<TakeDataset element_spec=(TensorSpec(shape=(256,), dtype=tf.float64, name=None), TensorSpec(shape=(256,), dtype=tf.float64, name=None), TensorSpec(shape=(2,), dtype=tf.float64, name=None))>

In [13]:
def SentimentDatasetMapFunction(input_ids, attn_masks, labels):
    return {
        'input_ids': input_ids,
        'attention_mask': attn_masks
    }, labels

In [14]:
dataset = dataset.map(SentimentDatasetMapFunction) # converting to required format for tensorflow dataset 

In [15]:
dataset = dataset.shuffle(10000).batch(16, drop_remainder=True) # batch size, drop any left out tensor

In [16]:
p = 0.8
train_size = int((len(df_seek)//16)*p) # for each 16 batch of data we will have len(df)//16 samples, take 80% of that for train.

In [17]:
train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size)

In [18]:
train_dataset

<TakeDataset element_spec=({'input_ids': TensorSpec(shape=(16, 256), dtype=tf.float64, name=None), 'attention_mask': TensorSpec(shape=(16, 256), dtype=tf.float64, name=None)}, TensorSpec(shape=(16, 2), dtype=tf.float64, name=None))>

In [19]:
from transformers import TFBertModel

In [20]:
model = TFBertModel.from_pretrained('bert-base-cased') # bert base model with pretrained weights

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [21]:
# defining 2 input layers for input_ids and attn_masks
input_ids = tf.keras.layers.Input(shape=(256,), name='input_ids', dtype='int32')
attn_masks = tf.keras.layers.Input(shape=(256,), name='attention_mask', dtype='int32')

bert_embds = model.bert(input_ids, attention_mask=attn_masks)[1] # 0 -> activation layer (3D), 1 -> pooled output layer (2D)
intermediate_layer = tf.keras.layers.Dense(512, activation='relu', name='intermediate_layer')(bert_embds)
output_layer = tf.keras.layers.Dense(2, activation='softmax', name='output_layer')(intermediate_layer) # softmax -> calcs probs of classes

sentiment_model = tf.keras.Model(inputs=[input_ids, attn_masks], outputs=output_layer)
sentiment_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  108310272   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 256,                                           

In [22]:
optim = tf.keras.optimizers.legacy.Adam(learning_rate=1e-5, decay=1e-6)
loss_func = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')

In [23]:
sentiment_model.compile(optimizer=optim, loss=loss_func, metrics=[acc])

In [30]:
hist = sentiment_model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=2
)

Epoch 1/2
249/249 [==============================] - 327s 1s/step - loss: 0.3121 - accuracy: 0.8647 - val_loss: 0.2832 - val_accuracy: 0.8770
Epoch 2/2
249/249 [==============================] - 326s 1s/step - loss: 0.2361 - accuracy: 0.9006 - val_loss: 0.1581 - val_accuracy: 0.9435


In [31]:
sentiment_model.save('sentiment_model_seek')

INFO:tensorflow:Assets written to: sentiment_model_seek/assets


INFO:tensorflow:Assets written to: sentiment_model_seek/assets


TRAINING RESPONSE SIDE

In [32]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

In [33]:
token = tokenizer.encode_plus(
    df_response['text'].iloc[0], 
    max_length=256, 
    truncation=True, 
    padding='max_length', 
    add_special_tokens=True,
    return_tensors='tf'
)

In [34]:
X_input_ids = np.zeros((len(df), 256))
X_attn_masks = np.zeros((len(df), 256))

In [35]:
def generate_training_data(df_balanced, ids, masks, tokenizer):
    for i, text in tqdm(enumerate(df_response['text'])):
        tokenized_text = tokenizer.encode_plus(
            text,
            max_length=256, 
            truncation=True, 
            padding='max_length', 
            add_special_tokens=True,
            return_tensors='tf'
        )
        ids[i, :] = tokenized_text.input_ids
        masks[i, :] = tokenized_text.attention_mask
    return ids, masks

In [36]:
X_input_ids, X_attn_masks = generate_training_data(df, X_input_ids, X_attn_masks, tokenizer)

0it [00:00, ?it/s]

In [37]:
labels = np.zeros((len(df_response), 2))
labels.shape

(5007, 2)

In [38]:
labels[np.arange(len(df_response)), df_response['label'].values] = 1 # one-hot encoded target te

In [39]:
# creating a data pipeline using tensorflow dataset utility, creates batches of data for easy loading...
dataset = tf.data.Dataset.from_tensor_slices((X_input_ids, X_attn_masks, labels))
dataset.take(1) # one sample data

<TakeDataset element_spec=(TensorSpec(shape=(256,), dtype=tf.float64, name=None), TensorSpec(shape=(256,), dtype=tf.float64, name=None), TensorSpec(shape=(2,), dtype=tf.float64, name=None))>

In [40]:
def SentimentDatasetMapFunction(input_ids, attn_masks, labels):
    return {
        'input_ids': input_ids,
        'attention_mask': attn_masks
    }, labels

In [41]:
dataset = dataset.map(SentimentDatasetMapFunction) # converting to required format for tensorflow dataset 

In [42]:
dataset = dataset.shuffle(10000).batch(16, drop_remainder=True) # batch size, drop any left out tensor

In [43]:
p = 0.8
train_size = int((len(df_response)//16)*p) # for each 16 batch of data we will have len(df)//16 samples, take 80% of that for train.

In [44]:
train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size)

In [45]:
train_dataset

<TakeDataset element_spec=({'input_ids': TensorSpec(shape=(16, 256), dtype=tf.float64, name=None), 'attention_mask': TensorSpec(shape=(16, 256), dtype=tf.float64, name=None)}, TensorSpec(shape=(16, 2), dtype=tf.float64, name=None))>

In [46]:
from transformers import TFBertModel

In [47]:
model = TFBertModel.from_pretrained('bert-base-cased') # bert base model with pretrained weights

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [48]:
# defining 2 input layers for input_ids and attn_masks
input_ids = tf.keras.layers.Input(shape=(256,), name='input_ids', dtype='int32')
attn_masks = tf.keras.layers.Input(shape=(256,), name='attention_mask', dtype='int32')

bert_embds = model.bert(input_ids, attention_mask=attn_masks)[1] # 0 -> activation layer (3D), 1 -> pooled output layer (2D)
intermediate_layer = tf.keras.layers.Dense(512, activation='relu', name='intermediate_layer')(bert_embds)
output_layer = tf.keras.layers.Dense(2, activation='softmax', name='output_layer')(intermediate_layer) # softmax -> calcs probs of classes

sentiment_model = tf.keras.Model(inputs=[input_ids, attn_masks], outputs=output_layer)
sentiment_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  108310272   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 256,                                         

In [49]:
optim = tf.keras.optimizers.legacy.Adam(learning_rate=1e-5, decay=1e-6)
loss_func = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')

In [50]:
sentiment_model.compile(optimizer=optim, loss=loss_func, metrics=[acc])

In [51]:
hist = sentiment_model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=2
)

Epoch 1/2


2023-02-14 00:21:32.413450: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


249/249 [==============================] - ETA: 0s - loss: 0.2824 - accuracy: 0.8783

2023-02-14 00:26:53.606760: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


249/249 [==============================] - 352s 1s/step - loss: 0.2824 - accuracy: 0.8783 - val_loss: 0.1800 - val_accuracy: 0.9296
Epoch 2/2
249/249 [==============================] - 327s 1s/step - loss: 0.1848 - accuracy: 0.9307 - val_loss: 0.1002 - val_accuracy: 0.9692


In [52]:
sentiment_model.save('sentiment_model_reponse')

INFO:tensorflow:Assets written to: sentiment_model_reponse/assets


INFO:tensorflow:Assets written to: sentiment_model_reponse/assets


In [53]:
test_seek = pd.read_csv("self_eval_seek_2.csv")
test_seek = test_seek[test_seek["seeking?"].notna()]
test_seek = test_seek.loc[:, ~test_seek.columns.str.contains('^Unnamed')]
test_seek = test_seek.drop_duplicates(subset='seeker_post', keep="last")
test_seek.loc[test_seek["seeking?"]=="Seeking(subtly)", "seeking?"] = int(0)
test_seek.loc[test_seek["seeking?"]=="Not Seeking", "seeking?"] = int(0)
test_seek.loc[test_seek["seeking?"]=="Not Seeking/Maybe", "seeking?"] = int(0)
test_seek.loc[test_seek["seeking?"]=="Seeking(truly)", "seeking?"] = int(1)
test_seek["seeking?"].value_counts()

1    126
0     75
Name: seeking?, dtype: int64

In [54]:
sentiment_model_seek = tf.keras.models.load_model('sentiment_model_seek')

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

def prepare_data(input_text, tokenizer):
    token = tokenizer.encode_plus(
        input_text,
        max_length=256, 
        truncation=True, 
        padding='max_length', 
        add_special_tokens=True,
        return_tensors='tf'
    )
    return {
        'input_ids': tf.cast(token.input_ids, tf.float64),
        'attention_mask': tf.cast(token.attention_mask, tf.float64)
    }

def predict_class(processed_data):
    '''predict class of input text
  Args:
    - reviews (list of strings)
  Output:
    - class (list of int)
  '''
    result = []
    for i in processed_data:
        pred = sentiment_model_seek.predict(i)
        result.append(np.argmax(pred))
    return result


In [55]:
from sklearn.metrics import classification_report

processed_data = []

for i in test_seek["seeker_post"]:
    processed_data.append(prepare_data(i, tokenizer))
y_pred = predict_class(processed_data)
print(y_pred)
print(classification_report(test_seek["seeking?"].to_list(), y_pred))

2023-02-14 00:33:47.192825: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 132ms/step
[0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1]
              precision    recall  f1-score   support

           0       0.66      0.76      0.70        75
           1       0.84      0.76      0.80       126

    accuracy                           0.76       201
   macro avg       0.75      0.76      0.75       201
weighted avg       0.77      0.76      0.76       201



In [3]:
test_response = pd.read_csv("Sharma_response_0_1.csv")
# test_response.loc[test_response["empathetic?"]=="Empathetic", "empathetic?"] = 1
# test_response.loc[test_seek["empathetic?"]=="Non empathetic", "empathetic?"] = 0
test_response = test_response.loc[:, ~test_response.columns.str.contains('^Unnamed')]
test_response

,response_post,level
0,"That's pretty vague, do you not know what you'...",0
1,idk what a Red pill means exactly but my advic...,0
2,"I think it's social anxiety , that creates par...",0
3,By any chance do you think you're in a loop. J...,0
4,depression. not sadness which is caused by som...,0
...,...,...
9247,"All day, every day. It's definitely not just y...",1
9248,like being stuck in a black hole. At times you...,0
9249,I probably would have considered bringing harm...,1
9250,People barely notice me too,1


In [4]:
sentiment_model_seek = tf.keras.models.load_model('sentiment_model_reponse')

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

def prepare_data(input_text, tokenizer):
    token = tokenizer.encode_plus(
        input_text,
        max_length=256, 
        truncation=True, 
        padding='max_length', 
        add_special_tokens=True,
        return_tensors='tf'
    )
    return {
        'input_ids': tf.cast(token.input_ids, tf.float64),
        'attention_mask': tf.cast(token.attention_mask, tf.float64)
    }

def predict_class(processed_data):
    '''predict class of input text
  Args:
    - reviews (list of strings)
  Output:
    - class (list of int)
  '''
    result = []
    for i in processed_data:
        pred = sentiment_model_seek.predict(i)
        result.append(np.argmax(pred))
    return result
#     print(sentiment_model_seek.predict(processed_data))
#     return [np.argmax(pred) for pred in sentiment_model_seek.predict(processed_data)]
#     probs = model.predict(processed_data)[0]
#     return classes[np.argmax(probs)]



Metal device set to: Apple M1 Pro


2023-02-14 01:33:35.195159: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-14 01:33:35.197562: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-02-14 01:33:35.197843: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [5]:
from sklearn.metrics import classification_report

processed_data = []

for i in test_response["response_post"]:
    processed_data.append(prepare_data(i, tokenizer))
# print(processed_data)
y_pred = predict_class(processed_data)
print(y_pred)
print(classification_report(test_response["level"].to_list(), y_pred))

2023-02-14 01:34:05.527054: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 153ms/step


1/1 [==============================] - 0s 153ms/step


1/1 [==============================] - 0s 154ms/step


1/1 [==============================] - 0s 162ms/step


...

1/1 [==============================] - 0s 157ms/step


1/1 [==============================] - 0s 159ms/step


1/1 [==============================] - 0s 156ms/step


1/1 [==============================] - 0s 170ms/step


1/1 [==============================] - 0s 184ms/step
[0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0